# 3. Pinecone のベクターDBを動かしてみよう

VectorDBという技術はLLMの出現以前から存在していたものですが、多くのLLMソリューションにとって重要な役割を果たします。特に、RAGによってユーザーのプロンプトにベクトルDBの検索結果を追加することで、ハルシネーションや長期記憶の問題に対処しする場合によく使われます。

[Pinecone](https://www.pinecone.io/) は Cloud ベースのベクターDBで、簡単に利用することができます。このノートブックでは、実際にPineconeを利用してみます。

前提として、Lab2で講師が特定の Web サイトをスクレイピングし、各ページの内容を Pinecone のベクターDBにロードしてあります。このLab (Lab3) では、Jupyter から作成済みの Pinecone にアクセスすることにフォーカスします。

![Exercise 3 overview](../assets/exercise_3.png)

### 3.1 インポートとグローバル変数の設定

In [1]:
import os
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL_REPO = "sentence-transformers/all-mpnet-base-v2"
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')
PINECONE_INDEX = os.getenv('PINECONE_INDEX')
dimension = 768 #ベクトルの次元数＝768 <- この次元数を、インデックスごとに揃えておく必要があります（同じ Embedding モデルを使っていれば、次元は揃います）

### 3.2 Pinecone とのコネクションの初期化
上記のセルで指定した値で、Pinecone のクライアントを初期化します。

In [2]:
print("Pinecone のコネクションを初期化中...")
pinecone = Pinecone(api_key=PINECONE_API_KEY)
print("Pinecone の初期化が完了しました")

print(f"Getting '{PINECONE_INDEX}' as object...")
index = pinecone.Index(PINECONE_INDEX)
print("Success")

# インデックスの最新情報を取得
current_collection_stats = index.describe_index_stats()
print('Pinecone のインデックスに入っているベクターの数は {} です。'.format(current_collection_stats.get('total_vector_count')))

initialising Pinecone connection...
Pinecone initialised
Getting 'llm-hol' as object...
Success
Total number of embeddings in Pinecone index is 15.


### 3.3 ベクトル検索を行う関数の定義

ベクトル検索の要点は、ユーザーのプロンプトに最も近いナレッジベースを探すことです。
ユーザーの質問をもとにセマンティックサーチを行い、ユーザーの質問に最も意味が近いナレッジベースを検索し、その内容をソースとスコアとともに返します。

In [3]:
# ユーザーの質問をエンべディングし、最も近いナレッジベースを Pinecone のベクターDBから検索する
def get_nearest_chunk_from_pinecone_vectordb(index, question):
    # エンべディングモデルを利用して、ユーザーの質問をエンべディングする
    retriever = SentenceTransformer(EMBEDDING_MODEL_REPO)
    xq = retriever.encode([question]).tolist()
    xc = index.query(vector=xq, top_k=5,
                 include_metadata=True)
    
    matching_files = []
    scores = []
    for match in xc['matches']:
        # メタデータの中のファイルパスを取得する
        file_path = match['metadata']['file_path']
        # 各ベクターのスコアを抽出する
        score = match['score']
        scores.append(score)
        matching_files.append(file_path)

    # 最も近いナレッジベースのテキストを返す
    # ここでは返却する結果は1件のみ
    response = load_context_chunk_from_data(matching_files[0])
    sources = matching_files[0] # matching_files には、マッチしたファイルが複数件入っているが、ここではトップの1件のみを抽出する。
    score = scores[0]
    return response, sources, score

# ナレッジベースのID（相対ファイルパス）を返却する
def load_context_chunk_from_data(id_path):
    with open(id_path, "r") as f: # Open file in read mode
        return f.read()

### 3.4 ベクトル検索を実行する

質問文の内容をもとにベクトル検索を行い、その結果をノートブック上で表示してみましょう。
ベクトル検索の結果を、メタデータとともに抽出できることに注意しておきましょう。
ここでは、ソースとなるテキストファイルのパスをメタデータとして保持しているため、検索範囲を絞ったり、回答が正しいかどうかを確認することができます。

※初回の実行には少し時間がかかります  
※実行中に表示されるバグレポートは一時的なものです

In [ ]:
question = "CMLってなんですか?" ## 聞きたい質問に置き換えてみましょう

context_chunk, sources, score = get_nearest_chunk_from_pinecone_vectordb(index, question)
print("\nContext Chunk: - 検索結果の本文")
print(context_chunk)
print("\nContext Source(s) - 検索結果のファイルのありか: ")
print(sources)
print("\nPinecone Score - 近似値: ")
print(score)

### 3.5 まとめ
* ベクトル検索は、RAGのアーキテクチャを利用するLLMにとって欠かせない構成要素です。
* Cloudera のパートナーである[Pinecone](https://www.pinecone.io/) は、SaaS ベースの手軽なベクターDBを提供しています。
* ベクトルDBにはメタデータを格納することができ、検索結果の検証や確認に利用することができます。

### 次のステップ：Lab5  - Chatbot のアプリを起動してみよう